<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1.  Installation Test
Run the following cell to attempt an import of the package and check if it is properly installed.

In [3]:
import fastoad

## 2. User files overview

The FAST-OAD environment uses two types of user files:

* Configuration files (**.toml**):

    The configuration example file used for the tutorial can be found [here](./workdir/oad_process.toml) (`api.generate_configuration_file()` must have been run - see Section 3).

    FASTOADProblem instances require a configuration file at initialization. The usage of this type of file is described in the [README](https://github.com/fast-aircraft-design/FAST-OAD) of the repository.

    
* System variables files (**.xml**):

    These files regroup the information of the variables involved in the system model. There are two types of system variables files:
    
    1 - INPUT_FILE: The input file contains the global inputs values required to run all the model. The user is free to modify the values of the variables in order that these new values are considered during a model run.
    
    2 - OUTPUT_FILE: The output file contains all the variables (inputs + outputs) values obtained after a model run.

The following figure shows an overview of the user files:

![User Files](./img/user_files_arch.svg "User Files Architecture")

The user defines the CONFIGURATION_FILE that the FASTOADProblem instance will read. The user can read and write this file, and thus modify the problem, the input and output file names etc. The FASTOADProblem then knows which is the structure of the model and what are the modules used. Thus, it is capable of determining which are the global inputs required for running the model. 

Executing the `generate_inputs()` function with the `configuration_file_path` as the first argument will generate the INPUT_FILE.xml file that contains all the global inputs. If a second argument `source_path` (SOURCE_PATH.xml) is provided then the instance will use the values found in this file for the same variables of the INPUT_FILE. If the SOURCE_PATh file uses a different schema, a translation file `source_path_schema` can be provided as a third argument to the function.

Once the variables of the INPUT_FILE have a correct value (by default values are NaN) the problem can be executed. For that the user can perform either a Multidisciplinary Design Analysis (MDA) using `evaluate_problem()` or a Multidisciplinary Design Optimization (MDO) using `optimize_problem()`. Both of these function generate the OUTPUT_FILE.xml containing all the variables of the system and their values resulting from the computation.

## 3. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders '/data' and '/workdir'. For instance, in '/data' we store a .xml file which describes the [CeRAS reference case](http://ceras.ilr.rwthaachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In '/workdir', we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging

DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.toml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

Unable to import mpi4py. Parallel processing unavailable.


After defining a configuration file name, we can ask FAST-OAD to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\oad_process.toml


You can now checkout the generated [configuration file](./workdir/oad_process.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in D:\s.delbecq\Documents\dev\FAST-OAD\notebooks\tutorial\workdir\problem_inputs.xml


You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [4]:
api.list_outputs(CONFIGURATION_FILE)

-- OUTPUTS OF THE PROBLEM ------------------------------------------------------------
VARIABLE                                                    | DESCRIPTION
geometry:cabin:NPAX1                                        | number of passengers if there are only economical class seats
cabin:Nrows                                                 | 
weight:systems:flight_kit:CG:x                              | flight kit (C6): X-position of center of gravity
weight:furniture:passenger_seats:CG:x                       | passenger seats (D2): X-position of center of gravity
cg_pl:CG_PAX                                                | 
geometry:fuselage:length                                    | total fuselage length
geometry:fuselage:maximum_width                             | maximum fuselage width
geometry:fuselage:maximum_height                            | maximum fuselage height
geometry:fuselage:front_length                              | length of front non-cylindrical part of the f

Another useful feature is to list the modules of the model defined in the configuration file:

In [5]:
api.list_systems(CONFIGURATION_FILE)

-- AVAILABLE SYSTEM IDENTIFIERS ------------------------------------------------------
IDENTIFIER                                                  | PATH
fastoad.aerodynamics.highspeed.legacy                       | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\aerodynamics\aerodynamics_high_speed.py
fastoad.aerodynamics.legacy                                 | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\aerodynamics\aerodynamics.py
fastoad.geometry.legacy                                     | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\geometry\geometry.py
fastoad.performances.breguet.from_mtow                      | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\performances\breguet.py
fastoad.performances.breguet.from_owe                       | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\performances\breguet.py
fastoad.propulsion.rubber_engine                            | d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [6]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
print(N2_FILE)
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in workdir\n2.html


workdir\n2.html


## 4. Running your first MDA

In [7]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)
print('Input max thrust: %.0f' % eval_problem['propulsion:MTO_thrust'], ' [N]')
print('Cruise thrust rate: %.2f' % eval_problem['propulsion:thrust_rate'], ' [-]')
print('MTOW: %.0f' % eval_problem['weight:aircraft:MTOW'], ' [kg]')

d:\s.delbecq\documents\dev\fast-oad\src\fastoad\modules\performances\breguet.py:235: RuntimeWarning:

overflow encountered in exp

INFO    : Computation finished. Problem outputs written in D:\s.delbecq\Documents\dev\FAST-OAD\notebooks\tutorial\workdir\problem_outputs.xml


NL: NLBGS Converged in 15 iterations
Input max thrust: 70000  [N]
Cruise thrust rate: 1.17  [-]
MTOW: 73824  [kg]


The problem has been solved, but you can in log messages and in outputs that the resulting thrust rate is above 1. **Indeed, the specified maximum thrust is too low.**

By the way, though there is a specified constraint in the configuration for having thrust rate between 0. and 1., this constraint does apply only in optimization problems.

You may run the problem again after having increased the maximum thrust, or run an optimization problem that will take it into account.

## 5. Running your first MDO

The configuration file defines a (not very relevant) optimization problem that aims at minimizing the MTOW of aircraft for the defined mission by adjusting the maximum thrust of engines (which has some influence on their fuel consumption, hence the needed block fuel).

*(This run should take a couple of minutes)*

In [9]:
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)
print('"Optimized" max thrust: %.0f' % optim_problem['propulsion:MTO_thrust'], ' [N]')
print('Cruise thrust rate: %.2f' % optim_problem['propulsion:thrust_rate'], ' [-]')
print('MTOW: %.0f' % optim_problem['weight:aircraft:MTOW'], ' [kg]')

NL: NLBGS Converged in 15 iterations


NL: NLBGS Converged in 5 iterations


NL: NLBGS Converged in 21 iterations


NL: NLBGS Converged in 1 iterations


NL: NLBGS Converged in 31 iterations


...

NL: NLBGS Converged in 4 iterations


NL: NLBGS Converged in 17 iterations


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.5501708394459386
            Iterations:

INFO    : Computation finished. Problem outputs written in D:\s.delbecq\Documents\dev\FAST-OAD\notebooks\tutorial\workdir\problem_outputs.xml


 3
            Function evaluations: 13
            Gradient evaluations: 3
Optimization Complete
-----------------------------------
"Optimized" max thrust: 107301  [N]
Cruise thrust rate: 0.80  [-]
MTOW: 76505  [kg]
